# Integration and calibration benchmarks

This benchmark requires the `integration.h5` file produced by `Preparation/integration_prep.ipynb`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from ctabench.visualization import plot_binned_mean

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"));

In [ ]:
filename = 'integration.h5'

In [ ]:
integrators = ['NeighbourPeakIntegrator',
               'LocalPeakIntegrator',
               'FullIntegrator']

cam_ids = ['LSTCam', 'NectarCam', 'FlashCam']


integration_data = {}

for integrator in integrators:
    integration_data[integrator] = {}
    for cam_id in cam_ids:
        integration_data[integrator][cam_id] = pd.read_hdf(filename, key=integrator+'/'+cam_id)

# Correlation of signal with true pe signal

In [ ]:
for integrator in integration_data.keys():
    print(integrator)
    fig, axes = plt.subplots(1, 2, figsize=(25,10))
    for ii, cam_id in enumerate(integration_data[integrator].keys()):
        signal_sum = integration_data[integrator][cam_id].calibrated_signal_sum
        true_pe_sum = integration_data[integrator][cam_id].true_pe_sum
        mc_energy = integration_data[integrator][cam_id].energy
        filter = true_pe_sum < 1000

        reg = sns.regplot(true_pe_sum[filter], 
                          signal_sum[filter], 
                          label=cam_id+'_'+integrator,
                          scatter_kws={'alpha':0.5},
                          ax=axes[0],
                          color=plt.cm.Dark2(ii),
                         )
#         reg.axes.set_xlim(0, 600)
#         reg.axes.set_ylim(0, 600)

        plot_binned_mean(np.log10(mc_energy[filter]), 
                         np.abs(true_pe_sum - signal_sum)[filter], 
                         color=plt.cm.Dark2(ii),
                         label=cam_id+'_'+integrator,
                         ax=axes[1],
                        )

    
    
    axes[0].plot(true_pe_sum[filter], true_pe_sum[filter], color='black')
    
    axes[1].set_xlabel("log10(E/TeV)")
    axes[1].set_ylabel("calibrated_pe - true_pe")
    axes[0].set_title("Calibrated pe as a function of true pe")
    axes[1].set_title("Distance from calibrated to true pe as a function of true pe")
    axes[0].legend()
    axes[1].legend()
    plt.show()